# Semantic Kernel 1.x Agent

This notebook demonstrates a **Semantic Kernel 1.x** agent built with the
Python `ChatCompletionAgent`.

The agent:

1. Takes a US company name from the user.
2. Uses the OpenAI model to infer the primary US stock ticker.
3. Calls a `MarketDataPlugin` tool to fetch ~10 years of daily prices from Stooq
   and compute which weekday has the lowest average closing price.
4. Calls an `OutlookPlugin` tool to create (or simulate) an Outlook reminder on that weekday next week.
5. Explains to the user what it did.

All of this is orchestrated by one `ChatCompletionAgent`** with automatic
function calling enabled by default in Semantic Kernel 1.x.

In [2]:
# If needed, install dependencies in this environment:
# !pip install semantic-kernel pywin32 requests

import os
from datetime import datetime, timedelta
from typing import Optional
from dotenv import load_dotenv
import requests

from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

try:
    import win32com.client as win32
except ImportError:
    win32 = None   # Outlook support disabled

import csv
from collections import defaultdict
import asyncio

In [3]:
# ==== Configuration ====

# Load environment variables from .env if present
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Use an OpenAI chat model you have access to.
CHAT_MODEL = "gpt-4.1-mini"  

In [4]:
class MarketDataPlugin:
    """Tool: analyze ~10 years of daily prices via Stooq and find best weekday.
    A simple CSV download from Stooq (https://stooq.com/).
    """

    @kernel_function(
        name="analyze_best_weekday",
        description=(
            "Given a US stock ticker (like F or MSFT), download the last 10 years "
            "of daily prices from Stooq and return which weekday has the lowest "
            "average closing price, with a short explanation."
        ),
    )
    def analyze_best_weekday(self, us_ticker: str) -> str:
        symbol = us_ticker.lower() + ".us"  # Stooq US suffix
        url = f"https://stooq.com/q/d/l/?s={symbol}&i=d"
        resp = requests.get(url, timeout=20)
        resp.raise_for_status()

        lines = resp.text.strip().splitlines()
        if len(lines) <= 1:
            return f"No historical data found on Stooq for {us_ticker}."

        reader = csv.DictReader(lines)
        weekday_sums = defaultdict(float)
        weekday_counts = defaultdict(int)
        ten_years_ago = datetime.now() - timedelta(days=365 * 10)
        
        for row in reader:
            try:
                date = datetime.fromisoformat(row["Date"])  # naive date
                close = float(row["Close"])
            except Exception:
                continue

            if date < ten_years_ago:
                continue

            weekday_name = date.strftime("%A")
            weekday_sums[weekday_name] += close
            weekday_counts[weekday_name] += 1

        if not weekday_counts:
            return f"No usable daily data for {us_ticker} in the last 10 years."

        avg_by_weekday = {
            w: weekday_sums[w] / weekday_counts[w] for w in weekday_counts
        }

        best_weekday = min(avg_by_weekday, key=avg_by_weekday.get)
        best_avg = avg_by_weekday[best_weekday]

        summary_lines = [
            f"Analyzed last ~10 years of daily prices for {us_ticker} via Stooq.",
            "Average close by weekday:",
        ]
        for w, avg in sorted(avg_by_weekday.items()):
            summary_lines.append(f"- {w}: {avg:.2f}")

        summary_lines.append(
            f"\nThe weekday with the lowest average close is **{best_weekday}** (~{best_avg:.2f})."
        )

        return "\n".join(summary_lines)

In [5]:
class OutlookPlugin:
    """Tool: create (or simulate) an Outlook reminder next week on a weekday."""

    @kernel_function(
        name="create_reminder_for_weekday",
        description="Create an Outlook reminder next week on the given weekday to check a stock."
    )
    def create_reminder_for_weekday(
        self,
        weekday_name: str,
        stock_ticker: str,
        note: Optional[str] = None,
    ) -> str:
        weekday_name = weekday_name.strip().capitalize()
        weekdays = [
            "Monday","Tuesday","Wednesday",
            "Thursday","Friday","Saturday","Sunday"
        ]

        if weekday_name not in weekdays:
            return f"'{weekday_name}' is not a valid weekday name."

        today = datetime.now()
        target_index = weekdays.index(weekday_name)
        delta_days = (target_index - today.weekday() + 7) % 7
        if delta_days == 0:
            delta_days = 7
        target_date = today + timedelta(days=delta_days)
        target_date = target_date.replace(hour=9, minute=0, second=0, microsecond=0)

        # FIX: use win32 instead of win32com
        if win32 is None:
            return (
                f"(Simulation) Would create Outlook reminder for {stock_ticker} on "
                f"{target_date:%Y-%m-%d} ({weekday_name}). "
                "Install pywin32 for real Outlook reminders."
            )

        # Real Outlook reminder
        outlook = win32.Dispatch("Outlook.Application")
        appt = outlook.CreateItem(1)
        appt.Subject = f"Check stock {stock_ticker}"
        appt.Start = target_date
        appt.Duration = 15
        appt.Body = note or f"Check stock {stock_ticker}."
        appt.ReminderSet = True
        appt.ReminderMinutesBeforeStart = 15
        appt.Save()

        return (
            f"Created Outlook reminder for {stock_ticker} on "
            f"{target_date:%Y-%m-%d} ({weekday_name})."
        )

In [6]:
# ==== Build the Semantic Kernel ChatCompletionAgent ====

# Configure the OpenAI chat completion service for the agent
chat_service = OpenAIChatCompletion(
    ai_model_id=CHAT_MODEL,
    api_key=OPENAI_API_KEY,
)

# Agent instructions act as the "system prompt" / persona
AGENT_INSTRUCTIONS = (
    "You are an AI financial assistant. "
    "When the user gives you a US company name, you must: \n"
    "1) Use your own knowledge (no external ticker APIs) to infer its primary US stock ticker symbol.\n"
    "2) Call the market_tools.analyze_best_weekday tool with that ticker to analyze the last ~10 years "
    "of daily prices and find the weekday with the lowest average close.\n"
    "3) Call the outlook_tools.create_reminder_for_weekday tool to create an Outlook reminder next week "
    "on that weekday, to check the stock.\n"
    "4) Finally, explain to the user what you did, including which ticker you used, which weekday had the "
    "lowest average close, and what reminder was or would be created.\n"
    "Always follow that logical order and ALWAYS use the tools instead of doing market calculations yourself."
)

agent = ChatCompletionAgent(
    service=chat_service,
    name="StockReminderAgent",
    instructions=AGENT_INSTRUCTIONS,
    # Passing plugin instances exposes their @kernel_function methods as tools.
    plugins=[MarketDataPlugin(), OutlookPlugin()],
)

print("Agent initialized with OpenAI + plugins (market_tools, outlook_tools).")

Agent initialized with OpenAI + plugins (market_tools, outlook_tools).


In [7]:
async def run_agent_once(company_name: str) -> str:
    """Send one request to the agent for a given company name and return its response text."""

    user_message = f"Please run the full analysis for company: {company_name}."\
                   " Follow your instructions step by step."

    # ChatCompletionAgent in SK 1.x provides a `get_response` helper for single-turn calls.
    response = await agent.get_response(messages=user_message)
    return str(response.content)

In [17]:
# Run the agent once for a chosen company 
company_input = input("Enter a US company name (e.g., Ford): ")
final_response = await run_agent_once(company_input)

print("\n===== Agent Response =====\n")
print(final_response)

Enter a US company name (e.g., Ford):  NVIDIA



===== Agent Response =====

I identified NVIDIA's primary US stock ticker as NVDA. After analyzing the last ~10 years of daily prices, I found that Thursday has the lowest average closing price for NVDA, at about 33.94. I then created an Outlook reminder for you on the upcoming Thursday to check NVIDIA's stock performance. Let me know if you need anything else!
